# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [172]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

import warnings
warnings.filterwarnings("ignore")

In [173]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carnarvon,-24.8667,113.6333,19.40,75,28,4.53,AU,1691615377
1,1,la passe,45.5549,-0.8967,26.13,49,1,5.83,FR,1691615644
2,2,vilyuysk,63.7553,121.6247,13.41,60,1,3.29,RU,1691615645
3,3,polyarnyy,69.1989,33.4478,19.08,73,0,2.85,RU,1691615645
4,4,tolanaro,-25.0319,46.9987,19.42,90,6,3.03,MG,1691615647


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [179]:
# Configure the map plot
coordinates = {
    "Latitude": city_data_df["Lat"],
    "Longitude": city_data_df["Lng"],
    "City": city_data_df["City"],
    "Country": city_data_df["Country"],
    "Humidity": city_data_df["Humidity"]
}

coordinates_df = pd.DataFrame(coordinates)
map_plot = coordinates_df.hvplot.points("Longitude",
                                        "Latitude", 
                                        geo = True, 
                                        size = "humidity", 
                                        scale = 0.5, 
                                        tiles = "OSM")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [175]:
# Narrow down cities that fit criteria and drop any results with null values
vacay_df = coordinates_df[coordinates_df["Humidity"] < 70]

# Drop any rows with null values
vacay_df.dropna(inplace = True)

# Display sample data
vacay_df.sample(10)

,Latitude,Longitude,City,Country,Humidity
528,44.8528,65.5092,kyzylorda,KZ,50
276,-9.6608,20.3916,saurimo,AO,29
169,33.1282,-91.9612,crossett,US,28
278,19.6175,37.2164,port sudan,SD,30
219,-8.0089,-34.8553,olinda,BR,69
322,18.7000,-103.6583,coahuayana de hidalgo,MX,64
348,-16.4758,-43.4883,francisco sa,BR,43
386,-23.9000,-61.8500,ingeniero guillermo n. juarez,AR,34
271,30.9236,66.4512,chaman,PK,14
147,64.5667,143.2000,ust-nera,RU,60


### Step 3: Create a new DataFrame called `hotel_df`.

In [176]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacay_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.sample(10)

,Latitude,Longitude,City,Country,Humidity,Hotel Name
400,35.0465,-81.9818,boiling springs,US,47,
223,-21.0595,164.8658,kone,NC,69,
204,45.6113,0.0995,la couronne,FR,47,
439,42.3414,78.0056,kyzyl-suu,KG,46,
565,51.2167,-102.4677,yorkton,CA,52,
353,38.4774,47.0699,ahar,IR,48,
374,50.1834,-105.4011,levuka,CA,67,
493,-32.3567,22.5830,beaufort west,ZA,53,
133,-37.0676,-12.3116,edinburgh of the seven seas,SH,60,
278,19.6175,37.2164,port sudan,SD,30,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [177]:
# Set parameters to search for a hotel
radius = 1000
params = {
    "categories": "accommodation.hotel",
    "limit": 20,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df["Latitude"][index]
    long = hotel_df["Longitude"][index]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.sample(10)

Starting hotel search
la passe - nearest hotel: No hotel found
vilyuysk - nearest hotel: Мария
crane - nearest hotel: No hotel found
nurlat - nearest hotel: Гостиница «Нурлат»
buka - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
quatre cocos - nearest hotel: No hotel found
vuktyl - nearest hotel: Вуктыл
chibougamau - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
adamstown - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
berdigestyakh - nearest hotel: No hotel found
aswan - nearest hotel: Yaseen hotel
bethel - nearest hotel: No hotel found
kapuskasing - nearest hotel: Travelodge
puerto baquerizo moreno - nearest hotel: Dory's House
keflavik - nearest hotel: Núpan Deluxe
port mathurin - nearest hotel: Escale Vacances
talnakh - nearest hotel: No hotel found
moapa valley - nearest hotel: No hotel found
zhezqazghan - nearest hotel: No hotel

brisas barra de suchiate - nearest hotel: No hotel found
khandyga - nearest hotel: No hotel found
punta alta - nearest hotel: Gran Hotel
kyzylorda - nearest hotel: No hotel found
shihezi - nearest hotel: 石河子宾馆
korla - nearest hotel: Silver Star Hotel
fonte boa - nearest hotel: No hotel found
les cayes - nearest hotel: No hotel found
baker city - nearest hotel: Geiser Grand Hotel
abadeh - nearest hotel: No hotel found
ola - nearest hotel: No hotel found
agramunt - nearest hotel: No hotel found
birjand - nearest hotel: No hotel found
callao - nearest hotel: Casa Ronald
cuyamel - nearest hotel: No hotel found
brokopondo - nearest hotel: No hotel found
sechura - nearest hotel: Hospedaje Claritza
borba - nearest hotel: No hotel found
yorkton - nearest hotel: No hotel found


,Latitude,Longitude,City,Country,Humidity,Hotel Name
7,40.8108,69.1986,buka,UZ,31,No hotel found
330,43.5708,104.4250,dalandzadgad,MN,51,Хан Уул буудал
547,-12.0667,-77.1500,callao,PE,68,Casa Ronald
471,46.4547,-90.1710,ironwood,US,53,No hotel found
342,-9.2000,25.8500,bukama,CD,31,No hotel found
57,41.3712,-73.4140,bethel,US,52,No hotel found
265,47.0930,81.6294,urzhar,KZ,58,Sharikan
541,44.7749,-117.8344,baker city,US,41,Geiser Grand Hotel
250,23.9065,42.9172,afif,SA,11,No hotel found
218,52.2667,79.1667,klyuchi,RU,46,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [178]:
# Configure the map plot
map_plot2 = hotel_df.hvplot.points("Longitude",
                                   "Latitude",
                                   geo = True,
                                   size = "Humidity",
                                   hover_cols = ["Hotel Name", "Country"],
                                   scale = 0.5,
                                   tiles = "OSM")

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]   (Humidity,Hotel Name,Country)